# Description

This notebook shows a way to solve an inverse problem parameterized with a single quantity.

# Load libraries

In [ ]:
import numpy as np
import igl
import meshplot as mp
import time

import sys as _sys
_sys.path.append("../src")
from fem_system import *
from elastic_energy import *
from stretch_utils import *
from matplotlib import gridspec
import matplotlib.pyplot as plt

shadingOptions = {
    "flat":True,
    "wireframe":False,   
}

rot = torch.tensor(
    [[1.0,  0.0, 0.0],
     [0.0,  0.0, 1.0],
     [0.0, -1.0, 0.0]]
)

# Load mesh

We will use the ball for the current experiment.

In [ ]:
v, _, _, t, _, _ = igl.read_obj("../data/ball.obj")
v = torch.tensor(v)
t = torch.tensor(t)

be = igl.edges(igl.boundary_facets(to_numpy(t)))
bv = np.unique(igl.boundary_facets(to_numpy(t)))
e  = igl.edges(to_numpy(t))

aabb = torch.max(v, dim=0).values - torch.min(v, dim=0).values
length_scale = torch.mean(aabb)

# p = mp.plot(to_numpy(v @ rot.T), to_numpy(t), shading=shadingOptions)

# Linear/Non-Linear Elastic Solid

## Instantiation

We first specify the elasticity model to use for the elastic solid, as well as pinned vertices, and volumetric forces.

In [ ]:
rho     = 131.0  # [kg.m-3]
young   = 7.0e5 # [Pa] 
poisson = 0.2
force_mass = torch.zeros(size=(3,))
force_mass[2] = - rho * 9.81

# Determines the pinned indices and the lowest pinned indext
maxZ    = torch.max(v[:, 2])
pin_idx = list(torch.arange(v.shape[0])[v[:, 2] > maxZ - 0.1*aabb[2]])
lowest_pin_idx = torch.argsort(v[:, 2])[-len(pin_idx)-1]

ee    = NeoHookeanElasticEnergy(young, poisson)
solid = FEMSystem(v, t, ee, rho=rho, pin_idx=pin_idx, f_mass=force_mass)

## Manually search for a solution

In [ ]:
n_search = 50
n_steps  = 20
thresh   = 1.0

v_rest_init = v.clone()
v_rest_tmp  = v.clone()
v_target    = v.clone()[bv]

stretches = np.linspace(0.55, 1.1, n_search)

list_v_rest, list_v_eq, target_closeness = report_stretches(solid, v_rest_init, bv, v_target, stretches, lowest_pin_idx)


In [ ]:
plt.plot(stretches, target_closeness)
plt.title("Closeness to target", fontsize=14)
plt.xlabel("Stretch along $z$", fontsize=12)
plt.ylabel("Objective", fontsize=12)
plt.grid()


# Plot best solution

In [ ]:
min_stretch = stretches[torch.argmin(target_closeness)]

stretch_diag = torch.tensor([1.0, 1.0, min_stretch])
v_rest_opt  = list_v_rest[torch.argmin(target_closeness)]
v_eq_opt    = list_v_eq[torch.argmin(target_closeness)]

p = mp.plot(to_numpy(v_eq_opt @ rot.T), to_numpy(t), shading=shadingOptions)
p.add_points(to_numpy(v_eq_opt[solid.pin_idx, :] @ rot.T), shading={"point_color":"black", "point_size": 0.1 * length_scale})
p.add_edges(to_numpy(v_rest_opt @ rot.T), be, shading={"line_color": "blue"})
p.add_edges(to_numpy(v_target @ rot.T), be, shading={"line_color": "red"})